###url 가져오기

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
import re
import time
import pandas as pd

C:\Users\wmk51\AppData\Local\Temp\ipykernel_7068\835174478.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
chrome_path = "C:\\Users\\wmk51\\python-project\\chromedriver.exe"
options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
driver = webdriver.Chrome(options=options)
url = "https://www.musinsa.com/search/musinsa/goods?q=%EB%8B%88%ED%8A%B8&list_kind=small&sortCode=pop&sub_sort=&page=1&display_cnt=0&saleGoods=&includeSoldOut=&setupGoods=&popular=&category1DepthCode=&category2DepthCodes=&category3DepthCodes=&selectedFilters=&category1DepthName=&category2DepthName=&brandIds=&price=&colorCodes=&contentType=&styleTypes=&includeKeywords=&excludeKeywords=&originalYn=N&tags=&campaignId=&serviceType=&eventType=&type=&season=&measure=&openFilterLayout=N&selectedOrderMeasure=&shoeSizeOption=&d_cat_cd=&attribute=&plusDeliveryYn="
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()
WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.CLASS_NAME, "lcWnUT"))).click()
time.sleep(1)

In [4]:
html_url = driver.page_source
soup_url = BeautifulSoup(html_url,"html.parser")
time.sleep(1)
product_url_list=[]
url_list = soup_url.find_all("a" , attrs={"class" : "img-block"})[:10]

for item in  url_list:
    product_url_list.append(item["href"])

In [6]:
num = 0
for product_url in product_url_list[:5]:
    
    driver.get(product_url)
    driver.set_window_size(1920,1080)
    driver.maximize_window()
        
        
    ##최신 순으로 댓글 정렬    
    html_url = driver.page_source
    soup_url = BeautifulSoup(html_url,"html.parser")
    time.sleep(2)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "reviewSelectSort"))).click()
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//option[@value="new"]'))
    ).click()
    
    num+=1
    html_product = driver.page_source
    soup_product = BeautifulSoup(html_product,"html.parser")
    time.sleep(1)
    #상품 이름
    prod_name = soup_product.find("h3",attrs={"class":"product-detail__sc-1klhlce-3"}).get_text()
    a = True
    i=0
    date_list=[]
    rating_list=[]
    reviewer_id_list = []
    tag_list = []
    print(num,"번쨰 상품")
    while a:
        ## i를 5로 나눠서 나머지가 0이면 next버튼을 눌러준다
        
        i+=1
        if( i % 5 == 0):
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".fa.fa-angle-right.paging-btn.btn.next"))).click()
            time.sleep(2)
            
        else:
            try:
                WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, f'//a[text()={i}]'))).click()
                time.sleep(2)
            except:
                break
        
        ##페이지 전환 후 다시 페이지 소스를 가져온다
        html_product = driver.page_source
        soup_product = BeautifulSoup(html_product,"html.parser")
        
        #작성일 가져오기
        #break문을 사용해서 확실히 끊어야 한다.
        
        dates = list(soup_product.find_all("p", attrs={"class":"review-profile__date"}))

        
        for date in dates:
            date = date.get_text()
            if '.' in date:
                a = False
                break
            date_list.append(date)
        
        if not a:  # 작성일을 가져오는 루프를 중단
            break
        
        ### 별점 가져오기
        # 5개는 100, 4개는 80, 3개는 60, 2개는 40, 1개는 20
        stars = soup_product.find_all("span", attrs={"class":"review-list__rating__active"})

        for star in stars:
            rating_list.append(re.sub(r'\D', '', star['style']))    
                
        # 리스트 안에 값을 int로 변환
        rating_list = [int(x) for x in rating_list]
        
        if not a:  # 작성일을 가져오는 루프를 중단
            break

        
        ### 작성자 가져오기
        
        
        reviewers= soup_product.find_all("p",attrs={"class" : "review-profile__name"})
        
        for reviewer in reviewers:
            reviewer_id_list.append(reviewer.get_text())
            
        
        if not a:  # 작성일을 가져오는 루프를 중단
            break
        
        print("review_id : ",len(reviewer_id_list),"rating : ",len(rating_list),"review_date : ",len(date_list))
        
        ##  소비자 해시태그 가져오기
        
        tags = soup_product.find_all("li",attrs={"class" : "review-evaluation--type3__item"})
        my_dict={}
        for tag in tags:
            
            my_dict={
                tag.get_text().split(" ")[0]:tag.get_text().split(" ")[1]
                
            }
            # 기간 내에 있는 모든 테그들을 담은 리스트
            tag_list.append(my_dict)
        
        
        
        

    # data_dict = {
    #                 "review_id" : reviewer_id_list,
    #                 "rating" : rating_list,
    #                 "review_date" : date_list
    #             }  
    # product_df = pd.DataFrame(data_dict)
    # product_df.to_csv(f'{prod_name}.csv', index=False,encoding='utf-8-sig') 
            
        
            
        
            
            
        
    
    


1 번쨰 상품
review_id :  10 rating :  10 review_date :  10
review_id :  20 rating :  20 review_date :  20
review_id :  30 rating :  30 review_date :  30
review_id :  40 rating :  40 review_date :  40
review_id :  50 rating :  50 review_date :  50
review_id :  60 rating :  60 review_date :  60
review_id :  70 rating :  70 review_date :  70
review_id :  80 rating :  80 review_date :  80
review_id :  90 rating :  90 review_date :  90
2 번쨰 상품
review_id :  10 rating :  10 review_date :  10
review_id :  20 rating :  20 review_date :  20
review_id :  30 rating :  30 review_date :  30
review_id :  40 rating :  40 review_date :  40
review_id :  50 rating :  50 review_date :  50
review_id :  60 rating :  60 review_date :  60
review_id :  70 rating :  70 review_date :  70
review_id :  80 rating :  80 review_date :  80
review_id :  90 rating :  90 review_date :  90
3 번쨰 상품
review_id :  4 rating :  4 review_date :  4
4 번쨰 상품
review_id :  10 rating :  10 review_date :  10
review_id :  20 rating :  20 re

KeyboardInterrupt: 

In [18]:

print(len(reviewer_id_list),len(rating_list))


10 10


In [11]:
reviewer_id_list = reviewer_id_list[:len(date_list)]
rating_list = rating_list[:len(date_list)]

print("작성자 : ",len(reviewer_id_list))
print("별점 : ",len(rating_list))
print("작성일 : ",len(date_list))

작성자 :  90
별점 :  90
작성일 :  90


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

C:\Users\wmk51\AppData\Local\Temp\ipykernel_3096\2151744951.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [13]:
data_dict = {

    "review_id" : reviewer_id_list,
    "rating" : rating_list,
    "review_date" : date_list
    
}

In [14]:
product_df = pd.DataFrame(data_dict)
product_df

,review_id,rating,review_date
0,LV.4 김토성,100,11시간 전
1,LV.5 아젤리아크림,100,22시간 전
2,LV.3 뉴비_2e262e2cccb7,100,1일 전
3,LV.5 뉴비_1f72d41a,100,1일 전
4,LV.6 김레온씨,100,1일 전
...,...,...,...
85,LV.6 실버_b568346580b9,100,27일 전
86,LV.5 Odleff,100,27일 전
87,LV.5 Odleff,100,27일 전
88,LV.3 로스차일드박,100,28일 전


In [ ]:
####여기서 부터는 태그 데이터  피처링

In [ ]:
tag_list

In [ ]:
data_dict = {
    '사이즈': ['보통이에요', '커요', '보통이에요', '보통이에요', '보통이에요'],
    '밝기': ['보통이에요', '보통이에요', '보통이에요', '밝아요'],
    '색감': ['선명해요', '선명해요', '보통이에요', '선명해요'],
    '두께감': ['보통이에요', '얇아요', '보통이에요', '보통이에요'],
    '촉감': ['부드러워요', '부드러워요', '평범해요'],
    '배송': ['빨라요', '빨라요', '빨라요'],
    '포장': ['꼼꼼해요', '꼼꼼해요', '꼼꼼해요']
}


In [27]:
from collections import defaultdict

# 빈도수를 저장할 딕셔너리
frequency = defaultdict(lambda: defaultdict(int))

data = [{'사이즈': '보통이에요'}, {'밝기': '보통이에요'}, {'색감': '선명해요'}, {'두께감': '보통이에요'}, {'촉감': '부드러워요'}, {'배송': '빨라요'}, {'포장': '꼼꼼해요'}, {'사이즈': '보통이에요'}, {'밝기': '보통이에요'}, {'색감': '선명해요'}, {'두께감': '보통이에요'}, {'촉감': '부드러워요'}, {'배송': '빨라요'}, {'포장': '꼼꼼해요'}, {'사이즈': '커요'}, {'밝기': '보통이에요'}, {'색감': '선명해요'}, {'두께감': '얇아요'}, {'촉감': '부드러워요'}, {'배송': '빨라요'}, {'포장': '꼼꼼해요'}, {'사이즈': '보통이에요'}, {'밝기': '보통이에요'}, {'색감': '보통이에요'}, {'두께감': '보통이에요'}]

# 각 키별로 값을 세기
for dictionary in tag_list:
    for key, value in dictionary.items():
        # print(key,value)
        # ex key가 사이즈면 해당 value값의 빈도가 올라감
        frequency[key][value] += 1
print(frequency)
key_list=[]
# 각 키별로 빈도가 가장 높은 값을 출력
for key, values in frequency.items():
    # print("key :" ,key,"value : ",value)
    max_value = max(values, key=values.get)
    #key = 사이즈 : 봉통이에요 - value["보통이에요"]
    print(f"{key}: {max_value} - {values[max_value]}번")
        

defaultdict(<function <lambda> at 0x0000023DD4495760>, {'사이즈': defaultdict(<class 'int'>, {'보통이에요': 95, '커요': 4, '작아요': 1}), '밝기': defaultdict(<class 'int'>, {'보통이에요': 80, '밝아요': 17, '어두워요': 3}), '색감': defaultdict(<class 'int'>, {'선명해요': 31, '보통이에요': 69}), '두께감': defaultdict(<class 'int'>, {'보통이에요': 87, '얇아요': 11, '두꺼워요': 2}), '촉감': defaultdict(<class 'int'>, {'부드러워요': 43, '평범해요': 57}), '배송': defaultdict(<class 'int'>, {'빨라요': 98, '아쉬워요': 2}), '포장': defaultdict(<class 'int'>, {'꼼꼼해요': 99, '아쉬워요': 1})})
사이즈: 보통이에요 - 95번
밝기: 보통이에요 - 80번
색감: 보통이에요 - 69번
두께감: 보통이에요 - 87번
촉감: 평범해요 - 57번
배송: 빨라요 - 98번
포장: 꼼꼼해요 - 99번
